In [ ]:
from csv import DictReader
import json

use csv.DictReader() for converting csv to dictionary and json for writing to json data files.

In [ ]:
from datetime import date, timedelta
import requests
import os

use datetime to generate the keys for the json file, requests to fetch the files from github, and os to delete data files